## Environment Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [13]:
import nest_asyncio

nest_asyncio.apply()

## Dataset

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/").load_data()

## Index Construction

In [38]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="12345678",
    url="bolt://localhost:7687",
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [71]:
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor


# What to extract
from typing import Literal
# recommended uppercase, underscore separated
entities = Literal["PERSON", "PLACE", "THING"]
relations = Literal["PART_OF", "HAS", "IS_A"]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

# Model to use
llm=OpenAI(model="gpt-4o-mini", temperature=0.0)


index = PropertyGraphIndex.from_documents(
    documents,
    llm=llm,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=llm,
            possible_entities=entities,
            possible_relations=relations,
            kg_validation_schema=schema,
        )
        
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 3/3 [00:02<00:00,  1.14it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, row) { ... }} {position: line: 10, column: 21, offset: 397} for query: "\n                    UNWIND $data AS row\n                    MERGE (e:__Node__ {id: row.id})\n                    SET e += apoc.map.clean(row.properties, [], [])\n                    SET e.name = row.name, e:`__Entity__`\n                    WITH e, row\n                    CALL apoc.create.addLabels(e, [row.label])\n                    YIELD node\n                    WITH e, row\n                    CALL {\n                        WITH e, row\n                        WITH e, row\n                        WHERE row.embedding IS NOT N

## Querying and Retrieval

In [72]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

In [73]:
query_engine = index.as_query_engine(include_text=True)

In [80]:
response = query_engine.query("list the requirements for visiting friend japan visa")

print(str(response))

The requirements for visiting a friend in Japan on a visa include:
1. Passport with holder's signature
2. Application Form with a facial photo attached
3. Certificate of Eligibility (if applicable)
4. Birth Certificate issued by PSA within 1 year
5. Marriage Certificate (if applicable)
6. Itinerary in Japan
7. Certificate of No Marriage Record (issued by PSA within 1 year)
8. Koseki-tohon of a Japanese parent (if applicable)
9. Photocopy of ID page and Japanese stamps in your old passport (if you visited Japan before)
10. Negative Certificate of Employment (if employed in the Philippines)
